In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from datetime import date
from datetime import datetime
import calendar
import os
import re


from scipy import stats
from sklearn.neighbors import KernelDensity
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation, PillowWriter

import seaborn as sns
from pathlib import Path
from tqdm import tqdm_notebook, trange, tqdm
from PIL import Image, ImageDraw
import io
import random as r

from matplotlib.widgets import Slider, Button, RadioButtons
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
# For downsampling
from sklearn.utils import resample,shuffle



In [2]:
df_companies = pd.read_csv("./data/companies.csv")
df_payments = pd.read_csv("./data/payments.csv")
df_physicians = pd.read_csv("./data/physicians.csv")

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
pd.set_option('display.max_columns', None)
df_physicians

,id,set,First_Name,Middle_Name,Last_Name,Name_Suffix,City,State,Zipcode,Country,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5
0,1,train,LEIGH,B,HOPPE,NaN,FORT PIERCE,FL,34950,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Obstetrics...,FL,NaN,NaN,NaN,NaN
1,2,train,STEVEN,NaN,FRANK,NaN,TOWSON,MD,21204-6808,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Anesthesio...,MD,NaN,NaN,NaN,NaN
2,3,train,THOMAS,NaN,COUCH,JR.,TROY,NY,12180-2832,UNITED STATES,NaN,Podiatric Medicine & Surgery Service Providers...,NY,NaN,NaN,NaN,NaN
3,4,train,CLAUDE,NaN,SU,NaN,SAVANNAH,GA,31419-1753,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Internal M...,GA,SC,AL,NaN,NaN
4,5,train,ROBERT,C,SCHENCK,NaN,ALBUQUERQUE,NM,87131-0001,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Orthopaedi...,NM,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,5996,train,MARC,NaN,WEINSTEIN,NaN,TEMPLE TERRACE,FL,33637-0925,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Orthopaedi...,FL,NaN,NaN,NaN,NaN
5996,5997,train,CHARLES,W,YOWELL,NaN,TALLAHASSEE,FL,32308-4620,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Urology,FL,SC,NaN,NaN,NaN
5997,5998,train,MARK,O,GABBIE,NaN,TEXARKANA,AR,71854,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Family Med...,AR,TX,NaN,NaN,NaN
5998,5999,test,KRISHNASWAMY,NaN,GAJARAJ,NaN,RANDOLPH,MA,02368-2100,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,MA,NaN,NaN,NaN,NaN


In [4]:
def phys_OI(phys_id):
    if(df_payments.query('Physician_ID==@phys_id and Ownership_Indicator=="Yes"').shape[0]>0):
        return 1
    else:
        return 0
    
df_physicians['Ownership_Interest']=df_physicians['id'].apply(phys_OI)


In [5]:
df_physicians.query('Ownership_Interest==1')

,id,set,First_Name,Middle_Name,Last_Name,Name_Suffix,City,State,Zipcode,Country,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5,Ownership_Interest
5,6,train,ARNOLD,NaN,TYRANGEL,NaN,BAYSIDE,NY,11360-2019,UNITED STATES,NaN,Dental Providers|Dentist|General Practice,NY,NaN,NaN,NaN,NaN,1
23,24,train,DOUGLAS,J,KIMBALL,NaN,BELGRADE,MT,59714-3915,UNITED STATES,NaN,Eye and Vision Services Providers|Optometrist,MT,NaN,NaN,NaN,NaN,1
32,33,train,GILBERT,A,LEIDIG,NaN,NEWARK,DE,19713-2137,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Internal M...,PA,DE,NaN,NaN,NaN,1
37,38,train,CURTIS,EARL,JANSEN,NaN,MONTEREY,CA,93940-2903,UNITED STATES,NaN,Dental Providers|Dentist|Prosthodontics,CA,NaN,NaN,NaN,NaN,1
68,69,train,LAURENCE,NaN,RUBIN,NaN,BETHPAGE,NY,11714,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Ophthalmology,NY,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5965,5966,train,EDGARDO,NaN,COLON ZAVALA,NaN,SAN JUAN,PR,00918-5032,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,PR,NaN,NaN,NaN,NaN,1
5968,5969,train,MAYO,A,NOERDLINGER,NaN,PORTSMOUTH,NH,03801-5679,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Orthopaedi...,IL,ME,NH,NaN,NaN,1
5980,5981,train,AHMAD,M,HOOSHMAND,NaN,JEFFERSON CITY,MO,65109-9508,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,MO,MA,NaN,NaN,NaN,1
5990,5991,train,MEREDITH,W,COUSIN,NaN,INDIANAPOLIS,IN,46237-8634,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Psychiatry...,IL,IN,NaN,NaN,NaN,1


In [6]:
df_companies.rename({"State":"State_Comp","Country":"Country_Comp"}, axis='columns', inplace=True)
df_physicians.rename({"State":"State_Phys","Country":"Country_Phys"}, axis='columns', inplace=True)
pd.set_option('display.max_columns', None)
df = pd.merge(df_payments, df_physicians, left_on='Physician_ID', right_on='id', how='left').drop('id', axis=1)
df = pd.merge(df, df_companies, on='Company_ID', how='left')
df.rename({"Name":"Company_Name"}, axis='columns', inplace=True)


###test safekeeping dataframe
df_main_test=df[df.set=="test"]

df = df.astype(str)
for column in df.columns:
     if(df[column].dtype=='object'):
        df[column] = df[column].str.lower()

#Convert to Date
df['Date']=pd.to_datetime(df['Date'])

#type conv after str conv
df[['Company_ID','Number_of_Payments']]=df[['Company_ID','Number_of_Payments']].astype(int)
df['Total_Amount_of_Payment_USDollars']=df['Total_Amount_of_Payment_USDollars'].astype(float)


In [7]:
###DROPPING RECORDS
df_scrap=df.query('Product_Code_1!="nan" and Product_Name_1=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)

df_scrap=df.query('Product_Code_2!="nan" and Product_Name_2=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)

df_scrap=df.query('Product_Code_3!="nan" and Product_Name_3=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)



In [8]:
###ENCODING CELL

name_list = pd.unique(df[['Product_Name_1', 'Product_Name_2', 'Product_Name_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Name_1'] = df['Product_Name_1'].apply(product_encode)
df['Product_Name_2'] = df['Product_Name_2'].apply(product_encode)
df['Product_Name_3'] = df['Product_Name_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Code_1', 'Product_Code_2', 'Product_Code_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Code_1'] = df['Product_Code_1'].apply(product_encode)
df['Product_Code_2'] = df['Product_Code_2'].apply(product_encode)
df['Product_Code_3'] = df['Product_Code_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Category_1'] = df['Product_Category_1'].apply(product_encode)
df['Product_Category_2'] = df['Product_Category_2'].apply(product_encode)
df['Product_Category_3'] = df['Product_Category_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Type_1', 'Product_Type_2','Product_Type_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Type_1'] = df['Product_Type_1'].apply(product_encode)
df['Product_Type_2'] = df['Product_Type_2'].apply(product_encode)
df['Product_Type_3'] = df['Product_Type_3'].apply(product_encode)


name_list = pd.unique(df[['License_State_1', 'License_State_2', 'License_State_3', 'License_State_4', 'License_State_5']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def license_encode(license_state):
    return name_list_dict.get(license_state)

df['License_State_1']=df['License_State_1'].apply(license_encode)
df['License_State_2']=df['License_State_2'].apply(license_encode)
df['License_State_3']=df['License_State_3'].apply(license_encode)
df['License_State_4']=df['License_State_4'].apply(license_encode)
df['License_State_5']=df['License_State_5'].apply(license_encode)

name_list = pd.unique(df[['Country_Comp', 'Country_Phys','Country_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Country_Comp'] = df['Country_Comp'].apply(product_encode)
df['Country_Phys'] = df['Country_Phys'].apply(product_encode)
df['Country_of_Travel'] = df['Country_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['State_Comp', 'State_Phys', 'State_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['State_Comp'] = df['State_Comp'].apply(product_encode)
df['State_Phys'] = df['State_Phys'].apply(product_encode)
df['State_of_Travel'] = df['State_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['City_of_Travel', 'City']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['City_of_Travel'] = df['City_of_Travel'].apply(product_encode)
df['City'] = df['City'].apply(product_encode)

df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "yes"),1,df.Ownership_Indicator)
df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "no"),0,df.Ownership_Indicator)
df['Ownership_Indicator']=df['Ownership_Indicator'].astype(int)

df['Date_year']=df['Date'].dt.year
df['Date_month']=df['Date'].dt.month
df['Date_day']=df['Date'].dt.day

columns_encode=['Nature_of_Payment_or_Transfer_of_Value','Form_of_Payment_or_Transfer_of_Value','Zipcode','Third_Party_Recipient', 'Charity', 'Third_Party_Covered','Contextual_Information', 'Related_Product_Indicator','Primary_Specialty']
for column in columns_encode:
    df[column]=df[column].astype('category')
    df[column]=df[column].cat.codes


In [9]:
###DROPPING COLUMNS
df_safe = df
df.drop(columns=['Date','Record_ID', 'First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 'Company_Name','Province'],inplace=True)
# df.drop(columns=['Date','Record_ID', 'First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 'Company_Name','Province','Product_Type_1','Product_Type_2','Product_Type_3','Product_Code_1','Product_Code_2','Product_Code_3'],inplace=True)

###SPLIT
df_test=df[df.set=="test"]
df_train=df[df.set=="train"]

###drop set column
df_test.drop(columns=['set'],inplace=True)
df_train.drop(columns=['set'],inplace=True)

##send predicted attribute to end
first_col = df_train.pop("Ownership_Interest")
df_train.insert((df_train.shape[1]),"Ownership_Interest",first_col)

first_col = df_test.pop("Ownership_Interest")
df_test.insert((df_test.shape[1]),"Ownership_Interest",first_col)

##send Physician ID to end
first_col = df_train.pop("Physician_ID")
df_train.insert((df_train.shape[1]),"Physician_ID",first_col)

first_col = df_test.pop("Physician_ID")
df_test.insert((df_test.shape[1]),"Physician_ID",first_col)



c:\python38\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [10]:
df_test

,Company_ID,Total_Amount_of_Payment_USDollars,Number_of_Payments,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Ownership_Indicator,Third_Party_Recipient,Charity,Third_Party_Covered,Contextual_Information,Related_Product_Indicator,Product_Code_1,Product_Code_2,Product_Code_3,Product_Type_1,Product_Type_2,Product_Type_3,Product_Name_1,Product_Name_2,Product_Name_3,Product_Category_1,Product_Category_2,Product_Category_3,City,State_Phys,Zipcode,Country_Phys,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5,State_Comp,Country_Comp,Date_year,Date_month,Date_day,Ownership_Interest,Physician_ID
2,760,10.26,1,3,6,1,32,26,0,2,0,0,29736,1,1,1,1,2,3,3,3,10,10,1,1,1,238,15,2805,1,120,3,53,53,53,53,3,1,2013,8,1,0,2632
10,760,10.26,1,3,6,1,32,26,0,2,0,0,29736,1,8,1,1,2,3,3,3,10,10,1,1,1,21,13,537,1,25,9,53,53,53,53,3,1,2013,8,1,0,2401
18,334,19.73,1,3,8,1,32,26,0,2,0,0,29736,1,13,1,1,2,3,3,15,10,10,1,1,1,3184,2,315,1,45,14,2,53,53,53,7,1,2013,8,1,0,3922
19,358,10.88,1,1,8,1,32,26,0,2,0,0,29736,1,14,1,1,2,3,3,16,10,10,1,1,1,1926,11,1973,1,37,15,53,53,53,53,8,1,2013,8,1,0,896
21,242,78.95,1,3,4,1,32,26,0,2,1,0,29736,1,16,1,1,2,3,3,18,10,10,1,1,1,1298,47,5352,1,16,16,53,53,53,53,10,1,2013,8,1,0,539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402236,74,14.89,1,3,8,1,32,26,0,2,0,0,29736,5,2043,1,1,5,3,3,6170,10,10,33,1,1,159,15,2748,1,16,3,53,53,53,53,2,1,2019,12,31,0,5046
1402238,700,35.26,1,1,8,1,32,26,0,2,0,0,29736,5,1,1,1,4,3,3,952,10,10,241,1,1,1305,21,3120,1,68,22,53,53,53,53,21,1,2019,12,31,0,473
1402240,836,52.91,1,3,8,1,32,26,0,2,0,0,29736,5,1,1,1,4,3,3,6293,10,10,1118,1,1,259,11,2118,1,71,15,29,53,53,53,3,1,2019,12,31,0,4940
1402244,381,4800.00,1,3,6,1,32,26,0,2,0,0,29736,5,1,1,1,4,3,3,1639,10,10,249,1,1,545,17,4432,1,113,13,53,53,53,53,13,1,2019,12,31,0,2965


In [ ]:
##heatmap cell
fig = plt.figure(figsize = (30,30))
ax = fig.gca()

heatmap = sns.heatmap(df_train[df_train.columns[0:41]].corr(), annot=True)
fig = heatmap.get_figure()
fig.savefig("heatmap_after_na_encode_OI.png")

In [ ]:
best=0
i=0
n=5
f = open("seeds.txt", "a")
f.write("File Starts here:\n")

while n>0:
    i+=1
    r_seed1=r.randint(0, 3000)
    r_seed2=r.randint(0, 3000)
    r_seed3=r.randint(0, 3000)
    r_seed4=r.randint(0, 3000)
    r_seed5=r.randint(0, 3000)
    n_est=r.randint(50,200)
    np.random.seed(r_seed1)
    ###downsampling
    df_train_maj=df_train[df_train.Ownership_Indicator==0]
    df_train_min=df_train[df_train.Ownership_Indicator==1]
    df_train_maj_down=resample(df_train_maj, replace=False, n_samples=df_train_min.shape[0], random_state=r_seed2)
    df_train_final = pd.concat([df_train_maj_down, df_train_min])

    ###test train split of own train set
    df_train_final['is_train']=np.random.uniform(0,1,len(df_train_final)) <=.80
    train, test = df_train_final[df_train_final['is_train']==True], df_train_final[df_train_final['is_train']==False]
    train = shuffle(train,random_state=r_seed3)
    test = shuffle(test,random_state=r_seed4)

    ###feature and label split
    features=train[train.columns[0:(train.shape[1]-3)]]
    y=train['Ownership_Indicator']
    x_test = test[test.columns[0:(test.shape[1]-3)]]
    y_test = test['Ownership_Indicator']

    ###model training
    clf = RandomForestClassifier(n_jobs=-1, oob_score = True, n_estimators = n_est, random_state=r_seed5)

    clf.fit(features, y)
    test_acc=clf.score(x_test,y_test)
#     print('Seed:'+str(r_seed1)+' '+str(r_seed2)+' '+str(r_seed3)+' '+str(r_seed4)+' '+str(r_seed5)+' '+str(test_acc))
    
    # Accuracy Scores
#     print ('Internal Accuracy Score', clf.oob_score_)
#     print ('RF accuracy: TRAINING', clf.score(features,y))
#     print ('RF accuracy: TESTING', test_acc)
    if test_acc>best:
        best=test_acc
        f.write('BEST:'+str(i)+' '+str(r_seed1)+' '+str(r_seed2)+' '+str(r_seed3)+' '+str(r_seed4)+' '+str(r_seed5)+' '+str(test_acc)+' N_EST: '+str(n_est)+'\n')
        print('BEST:'+str(i)+' '+str(r_seed1)+' '+str(r_seed2)+' '+str(r_seed3)+' '+str(r_seed4)+' '+str(r_seed5)+' '+str(test_acc)+' N_EST: '+str(n_est))
    score_test=clf.predict(x_test)
    if test_acc>0.9968814968814969:
        f.close()
        break

In [11]:
np.random.seed(2021)
###downsampling
df_train_maj=df_train[df_train.Ownership_Indicator==0]
df_train_min=df_train[df_train.Ownership_Indicator==1]
df_train_maj_down=resample(df_train_maj, replace=False, n_samples=df_train_min.shape[0], random_state=2021)
df_train_final = pd.concat([df_train_maj_down, df_train_min])

###test train split of own train set
df_train_final['is_train']=np.random.uniform(0,1,len(df_train_final)) <=.80
train, test = df_train_final[df_train_final['is_train']==True], df_train_final[df_train_final['is_train']==False]
train = shuffle(train,random_state=2021)
test = shuffle(test,random_state=2021)

In [14]:
print(train.shape)
train.columns[test.shape[1]-4]

(3839, 44)


'Date_day'

In [17]:
# 1261 533 2608 1581 2218 641 0.9968814968814969 N_EST: 63
# r_seed1, r_seed2, r_seed3, r_seed4, r_seed5, n_est= 473, 907, 1680, 322, 318, 142


###feature and label split
features=train[train.columns[0:(train.shape[1]-4)]]
y=train['Ownership_Interest']
x_test = test[test.columns[0:(test.shape[1]-4)]]
y_test = test['Ownership_Interest']

###model training
clf = RandomForestClassifier(n_jobs=-1, oob_score = True, n_estimators = 63, random_state=2021)
clf.fit(features, y)
test_acc=clf.score(x_test,y_test)

    # Accuracy Scores
print ('Internal Accuracy Score', clf.oob_score_)
print ('RF accuracy: TRAINING', clf.score(features,y))
print ('RF accuracy: TESTING', test_acc)
score_test=clf.predict(x_test)

Internal Accuracy Score 0.9619692628288616
RF accuracy: TRAINING 1.0
RF accuracy: TESTING 0.9690402476780186


In [18]:
###Creating Dataframes to find accuracy of Physician Level

finalAccPhysID=test['Physician_ID']
finalAcc={'Physician_ID':finalAccPhysID, 'Score':score_test}
finalAccDF=pd.DataFrame(data=finalAcc)
# finalAccDF['Physician_ID'].unique().size
physUFAD=finalAccDF['Physician_ID'].unique()
sc=[None]*physUFAD.size
for i in range(physUFAD.size):
    physician=physUFAD[i]
    if(finalAccDF.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0
        
finTestPredict={'Physician_ID':physUFAD, 'Score':sc}
finTestPredictDF=pd.DataFrame(data=finTestPredict)

TrueFinalAcc={'Physician_ID':finalAccPhysID, 'Score':test['Ownership_Indicator']}
TrueFinalAccDF=pd.DataFrame(data=TrueFinalAcc)
# finalAccDF['Physician_ID'].unique().size
physUFAD=TrueFinalAccDF['Physician_ID'].unique()
sc=[None]*physUFAD.size
for i in range(physUFAD.size):
    physician=physUFAD[i]
    if(TrueFinalAccDF.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0

TrueFinTest={'Physician_ID':physUFAD, 'Score':sc}
TrueFinTestDF=pd.DataFrame(data=TrueFinTest)




In [19]:
###Finding Accuracy
ownTestScore=TrueFinTestDF['Score']-finTestPredictDF['Score']
print(ownTestScore)
accuracy=(ownTestScore.value_counts(dropna=False,sort=False,ascending=True).loc[0])/ownTestScore.size*100
print(accuracy)

0      0
1      1
2      1
3      1
4      0
      ..
536    0
537    0
538    0
539    0
540    0
Name: Score, Length: 541, dtype: int64
72.2735674676525


In [ ]:
df_test
df_test.Physician_ID.unique().size

In [ ]:
print(features.columns)
print(df_test.columns)
predict_set=df_test
predict_set.drop(columns=['Ownership_Indicator','Physician_ID'],inplace=True)
print(features.shape)
print(predict_set.shape)

In [ ]:
score = clf.predict(predict_set)

In [ ]:
score.size

In [ ]:
phys_id_list=df_main_test['Physician_ID']

In [ ]:
submit={'Physician_ID':phys_id_list, 'Score':score}
submit_df=pd.DataFrame(data=submit)
submit_df['Physician_ID'].unique().size
physUnique=submit_df['Physician_ID'].unique()
physUnique.size
physUnique[0]

In [ ]:

sc=[None]*1000
for i in range(physUnique.size):
    physician=physUnique[i]
    if(submit_df.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0
        

In [ ]:
final_submit={'id':physUnique, 'prediction':sc}
fs_df=pd.DataFrame(data=final_submit)
fs_df

In [ ]:
fs_df.to_csv("./submission2.csv",index=False)

In [ ]:
print(physUnique)
print(sc)

In [ ]:
inspect1 = pd.read_csv("./submission2.csv")
inspect2 = pd.read_csv("./submission3.csv")
dfscrap=inspect2['prediction']-inspect1['prediction']
dfscrap.value_counts()